In [1]:
import pandas as pd
import os

In [29]:
df = pd.read_csv('Train_ULAK.csv').dropna()
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,68855579,10,6,1038,11595,346,0,103.800000,167.133879,...,32,998.0000,0.0000,998,998,6.830000e+07,0.0000,68300000,68300000,DoS Hulk
1,53,196,2,2,70,174,35,35,35.000000,0.000000,...,32,0.0000,0.0000,0,0,0.000000e+00,0.0000,0,0,BENIGN
2,123,118,2,2,96,96,48,48,48.000000,0.000000,...,20,0.0000,0.0000,0,0,0.000000e+00,0.0000,0,0,BENIGN
3,80,295657,7,10,1114,15841,1084,0,159.142857,407.829796,...,20,0.0000,0.0000,0,0,0.000000e+00,0.0000,0,0,BENIGN
4,80,15705351,7,5,407,452,377,0,58.142857,140.620563,...,20,360718.0000,0.0000,360718,360718,9.767208e+06,0.0000,9767208,9767208,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783351,41272,98689782,8,5,11595,327,5792,0,1449.375000,2046.673464,...,32,13859.0000,0.0000,13859,13859,9.860000e+07,0.0000,98600000,98600000,BENIGN
1783352,443,90956783,63,77,1041,135313,373,0,16.523810,68.018939,...,32,198255.7778,362537.3486,1165022,77291,9.908053e+06,290822.6482,10000000,9132848,BENIGN
1783353,443,1241813,3,3,0,0,0,0,0.000000,0.000000,...,28,0.0000,0.0000,0,0,0.000000e+00,0.0000,0,0,BENIGN
1783354,53,71509,2,2,78,330,39,39,39.000000,0.000000,...,32,0.0000,0.0000,0,0,0.000000e+00,0.0000,0,0,BENIGN


In [30]:
## Let's see if some assumption based on domain knowlwdge hold
# For this reason, let's split by label to start
gb=df.groupby(' Label')
dfs = [gb.get_group(x) for x in gb.groups]

In [31]:
import numpy as np
# This will let us know how to associate each group with a label
labels_per_group = [np.unique(df[[' Label']]) for df in dfs]
labels_per_group

[array(['BENIGN'], dtype=object),
 array(['Bot'], dtype=object),
 array(['DDoS'], dtype=object),
 array(['DoS GoldenEye'], dtype=object),
 array(['DoS Hulk'], dtype=object),
 array(['DoS Slowhttptest'], dtype=object),
 array(['DoS slowloris'], dtype=object),
 array(['FTP-Patator'], dtype=object),
 array(['Heartbleed'], dtype=object),
 array(['Infiltration'], dtype=object),
 array(['PortScan'], dtype=object),
 array(['SSH-Patator'], dtype=object),
 array(['Web Attack � Brute Force'], dtype=object),
 array(['Web Attack � Sql Injection'], dtype=object),
 array(['Web Attack � XSS'], dtype=object)]

# Level 1 filtering, the type of attack can only happen on particular destination port

## Let's check if web attacks happen only on ports 80 and 443

In [32]:
# Confirm the indeces for web attacks
labels_per_group[12:]

[array(['Web Attack � Brute Force'], dtype=object),
 array(['Web Attack � Sql Injection'], dtype=object),
 array(['Web Attack � XSS'], dtype=object)]

In [33]:
# In the training set only port 80 shows up
[np.unique(df[[' Destination Port']]) for df in dfs[12:]]

[array([80]), array([80]), array([80])]

## Lets check that SSH_Patator only happens on port 22

In [34]:
# Confirm the indeces for SSH-Patator
labels_per_group[11]

array(['SSH-Patator'], dtype=object)

In [35]:
# In the training set only port 22 shows up
[np.unique(df[[' Destination Port']]) for df in dfs[11:12]]

[array([22])]

## Let's check the FTP-Patator only happens on port 21

In [36]:
# Confirm the indeces for FTP-Patator
labels_per_group[7]

array(['FTP-Patator'], dtype=object)

In [37]:
# In the training set only port 21 shows up
[np.unique(df[[' Destination Port']]) for df in dfs[7:8]]

[array([21])]

# Let's start on Level 2 filter

## See how good the following rule works: 

BENIGN: Normal traffic

if ['Destination Port'] in [80, 443] and ['Flow Duration'] < threshold:
    return 'BENIGN'

In [38]:
# Confirm the indeces for Benign
labels_per_group[0]

array(['BENIGN'], dtype=object)

In [39]:
bdf = dfs[0]

In [7]:
p80fd = bdf[bdf[' Destination Port'] == 80][[' Flow Duration']]

In [40]:
p80fd.mean()

 Flow Duration    2.763582e+07
dtype: float64

In [41]:
p80fd.max()

 Flow Duration    119999979
dtype: int64

In [42]:
p80fd.std()

 Flow Duration    4.324367e+07
dtype: float64

In [43]:
ap80=dfs[1:]
ap80fd = [df[df[' Destination Port'] == 80][' Flow Duration'] for df in np80]

In [46]:
[df.mean() for df in ap80fd]

[nan,
 17073736.94560649,
 23386412.162399754,
 57109971.75181178,
 57551770.44659353,
 57075233.23938647,
 nan,
 nan,
 nan,
 739167.1764705882,
 nan]

In [47]:
[df.max() for df in ap80fd]

[nan,
 103941558,
 119311937,
 119611868,
 119800736,
 119046064,
 nan,
 nan,
 nan,
 9605093,
 nan]

In [48]:
[df.std() for df in ap80fd]

[nan,
 31142536.391154718,
 27532239.30393215,
 46017064.478114426,
 36274513.68910278,
 49442828.4695022,
 nan,
 nan,
 nan,
 1930486.2004055232,
 nan]